In [8]:
import pandas as pd

# read our file in 
rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet')

rides.head()

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


We want to create a new column that rounds to the previous hour since we will be working on an hourly basis. To do so, we can perform actions on datetime objects by using **.dt**. Once we use that, we can use **.floor** which rounds down and specify 'h' because we want to round the hour.

In [9]:

# add column of rounded hour
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('h')

In [10]:
# verify 
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


How many rides per location per hour? To get this information we need to use **.groupby**. We want to group our data so that we can see how many rides at each location happen per hour. Each row in our dataframe is a single ride that has the location ID and pickup hour included. This means there can be multiple rides to the same location per-hour. This information is beneficial to us because we can see how much demand we need for a given location in a given hour. 

After the data is grouped such that the pickup_location and pickup_hour have the same values, the **.size()** is used to create a new series that calculates the size, or number of occurences, of each group. Since this creates a series we use **.reset_index()** to convert it back to a data frame.

*be sure to use brackets [] in the groupby function since we are grouping by more than one column we use a list of columns.*

In [12]:
# create a column to count the number of rides per location per hour
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()

In [13]:
# view new data frame
agg_rides

,pickup_hour,pickup_location_id,0
0,2022-01-01 00:00:00,4,11
1,2022-01-01 00:00:00,7,6
2,2022-01-01 00:00:00,10,1
3,2022-01-01 00:00:00,12,2
4,2022-01-01 00:00:00,13,12
...,...,...,...
66863,2022-01-31 23:00:00,261,4
66864,2022-01-31 23:00:00,262,8
66865,2022-01-31 23:00:00,263,26
66866,2022-01-31 23:00:00,264,24


In [16]:
# rename the new column
agg_rides.rename(columns = {
    0: 'rides'
}, inplace = True)

In [17]:
agg_rides.head()

,pickup_hour,pickup_location_id,rides
0,2022-01-01,4,11
1,2022-01-01,7,6
2,2022-01-01,10,1
3,2022-01-01,12,2
4,2022-01-01,13,12


Next, we will make sure that each pickup hour is included in our data. For a large city like New York City, it is very unlikely that there are 0 taxi rides in a given hour but we still want to make sure that we are not excluding any hours from our data. If there are no taxi rides for a given location in a given hour we will impute a 0 so that we have no missing rows. This is done with a function and a for loop. Since for loops are very slow in Python, **tqdm** is imported which gives us a progress bar on the for loop.

In [19]:
from tqdm import tqdm

In [20]:
# a dataframe is given to this function that returns a dataframe
def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    # capture each location id using the .unique() function
    location_ids = agg_rides['pickup_location_id'].unique()

    # range of dates by hour
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(),
        agg_rides['pickup_hour'].max(), 
        freq='H')
    # empty dataframe
    output = pd.DataFrame()

    # loop through each unique location using tqdm to get a progress bar
    for location_id in tqdm(location_ids):

        # keep only rides for this 'location_id' and return a dataframe with only pickup_hour and rides columns
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]
            
        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        # set index to the pickup hour 
        agg_rides_i.set_index('pickup_hour', inplace=True)
        # make it datetime index
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        # should have the full range of hours and fill empty with 0 
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        
        # add back `location_id` columns since we did not have them in the for loop
        agg_rides_i['pickup_location_id'] = location_id

        # fill the empty dataframe we initiated
        output = pd.concat([output, agg_rides_i])
    
    # our index column is the pickup_hour - change the name
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    
    return output

In [22]:
# call the function
agg_rides_all_slots = add_missing_slots(agg_rides)

C:\Users\ryans\AppData\Local\Temp\ipykernel_37228\2872028829.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
100%|██████████| 257/257 [00:00<00:00, 303.30it/s]
